<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=05aeec65f28e07a65e4a7e74a6aa9e28fa503fb4337ff6496b85479d0d07ebfc
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-19 13:04:18
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.40 C
-------------------
Today PnL: 49.72 K (0.36%)
Current PnL: -27.35 L (-17.67%)
CY Booked + Current PnL: -13.06 L (-8.44%)
-------------------
Total profit:  1.33 L
Total loss:  -28.68 L
-------------------
Total Booked + Current PnL: 13.92 L (10.89%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.23%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 91.99 L (65.84%)
Deployed:  1.28 C
Current:  1.40 C
CAGR/XIRR %: 4.92%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-0.45,-20.50,25.91,0.09,20757.0,-20664.0,80113.0,85.91,26.0,M-SC,2.30,253.0,-1.00,0.58,5.25,OX40N,NTT,DURABLES
82,VOLTAS,1530.00,-2.28,7.15,11.70,19.69,24038.0,13713.0,205455.0,-3.29,50.0,X-MC,2.50,78.0,0.57,1.48,14.44,XY25,NTT,AC
50,MASFIN,398.61,1.39,-2.99,25.80,22.05,24524.0,-2925.0,95055.0,-16.44,52.0,H-SC,6.40,164.0,-0.12,0.68,38.17,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,0.11,-16.18,19.32,0.02,28269.0,-28239.0,146320.0,123.21,42.0,M-SC,13.10,234.0,-1.00,1.05,27.27,OX40N,NTT,PAINTS
43,ITC,452.00,0.37,-1.50,12.47,10.78,29519.0,-3599.0,236719.0,-41.15,47.0,X-LC,2.14,5.0,-0.12,1.70,3.66,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAELXSI,9161.0,6.93,-22.74,70.88,32.03,72198.0,-29976.0,101859.0,-16.37,60.0,H-SC,9.57,148.0,-0.42,0.73,14.09,OX40N,NTT,IT
6,ASIANTILES,137.0,5.13,-4.26,89.75,81.67,80611.0,-3993.0,89817.0,8083.33,71.0,L-SC,13.01,271.0,-0.05,0.65,73.93,XR,NTT,CERAMICS
10,BANDHANBNK,400.0,2.73,-28.18,167.93,92.42,335942.0,-78510.0,200049.0,40.80,50.0,H-SC,9.26,170.0,-0.23,1.44,15.61,XY24,NTT,BANKS
57,RAJOOENG,143.1,2.38,-35.73,117.21,39.61,77218.0,-36620.0,65880.0,-56.61,37.0,H-SC,30.38,135.0,-0.47,0.47,3.80,AR,ATH,MISC
55,QUESS,424.0,2.36,-29.06,101.36,42.84,46742.0,-18891.0,46115.0,-53.72,47.0,X-SC,34.74,83.0,-0.40,0.33,4.33,XY24,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VOLTAS,1530.00,-2.28,7.15,11.70,19.69,24038.0,13713.0,205455.0,-3.29,50.0,X-MC,2.50,78.0,0.57,1.48,14.44,XY25,NTT,AC
38,INDIAMART,4810.62,-1.84,-4.50,116.47,106.72,137181.0,-5554.0,117782.0,-52.73,37.0,H-SC,3.67,139.0,-0.04,0.85,17.82,AR,ATH,MISC
27,GLAXO,3466.20,-1.54,0.95,37.83,39.15,76108.0,1900.0,201184.0,-18.92,42.0,X-MC,10.51,60.0,0.02,1.45,30.23,X40,ATH,PHARMA
32,HCLTECH,1922.01,-1.32,6.39,17.24,24.73,44373.0,15462.0,257386.0,11.06,52.0,X-LC,6.90,7.0,0.35,1.85,22.60,X40,ATH,IT
48,KPIGREEN,730.66,-1.30,-16.13,75.22,46.95,79044.0,-20213.0,105084.0,-22.59,42.0,H-SC,8.98,154.0,-0.26,0.76,31.65,MH,ATH,POWER


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.82,-1.06,110.15,107.91,160902.0,-1569.0,146075.0,-19.34,59.0,M-SC,12.29,216.0,-0.01,1.05,8.92,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.11,-16.18,19.32,0.02,28269.0,-28239.0,146320.0,123.21,42.0,M-SC,13.10,234.0,-1.00,1.05,27.27,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.45,-20.50,25.91,0.09,20757.0,-20664.0,80113.0,85.91,26.0,M-SC,2.30,253.0,-1.00,0.58,5.25,OX40N,NTT,DURABLES
72,TATAELXSI,9161.00,6.93,-22.74,70.88,32.03,72198.0,-29976.0,101859.0,-16.37,60.0,H-SC,9.57,148.0,-0.42,0.73,14.09,OX40N,NTT,IT
66,SIS,528.00,0.71,-23.71,58.94,21.26,49929.0,-26321.0,84711.0,1998.96,53.0,H-SC,4.83,166.0,-0.53,0.61,14.45,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.11,8.33,59.01,72.27,108325.0,14122.0,183570.0,-6.99,69.0,M-LC,3.06,99.0,0.13,1.32,15.02,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.11,8.33,59.01,72.27,108325.0,14122.0,183570.0,-6.99,69.0,M-LC,3.06,99.0,0.13,1.32,15.02,XR,NTT,IT
25,FINCABLES,1641.55,0.82,-1.06,110.15,107.91,160902.0,-1569.0,146075.0,-19.34,59.0,M-SC,12.29,216.0,-0.01,1.05,8.92,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-1.84,-4.50,116.47,106.72,137181.0,-5554.0,117782.0,-52.73,37.0,H-SC,3.67,139.0,-0.04,0.85,17.82,AR,ATH,MISC
6,ASIANTILES,137.00,5.13,-4.26,89.75,81.67,80611.0,-3993.0,89817.0,8083.33,71.0,L-SC,13.01,271.0,-0.05,0.65,73.93,XR,NTT,CERAMICS
85,ZYDUSLIFE,1286.17,-0.09,-3.74,40.90,35.63,82512.0,-7834.0,201740.0,-17.18,36.0,H-MC,2.66,119.0,-0.09,1.45,12.31,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,PAGEIND,51605.08,0.13,-11.28,44.39,28.11,79325.0,-22710.0,178700.0,-35.73,24.0,X-MC,13.79,55.0,-0.29,1.29,0.13,X40,ATH,APPARELS
9,BAJAJHFL,181.50,0.16,-18.46,90.67,55.47,179868.0,-44910.0,198376.0,-27.82,27.0,X-MC,14.37,64.0,-0.25,1.43,1.64,X40N,ATH,FINANCE
8,AWL,485.00,0.25,-24.09,102.21,53.50,234119.0,-72694.0,229057.0,-63.25,29.0,X-MC,4.22,58.0,-0.31,1.65,1.74,XY24,NTT,FMCG
11,BATAINDIA,2096.00,0.35,-39.15,122.19,35.20,95672.0,-50372.0,78298.0,2.67,32.0,X-SC,16.62,92.0,-0.53,0.56,0.43,X40,NTT,FOOTWEAR
3,ACC,3906.00,0.22,-26.09,122.08,64.15,214714.0,-62071.0,175880.0,-56.75,32.0,X-SC,3.20,82.0,-0.29,1.27,0.22,XY24,BTT,CEMENT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,950.0,-0.39,-1.81,26.77,24.48,66203.0,-4554.0,247302.0,-81.32,44.0,X-SC,0.78,86.0,-0.07,1.78,20.91,X40N,NTT,MISC
12,BERGEPAINT,680.0,0.12,-4.33,26.63,21.14,57916.0,-9854.0,217485.0,-11.03,39.0,X-MC,1.33,74.0,-0.17,1.56,22.59,XY24,NTT,PAINTS
20,DABUR,735.0,0.09,-3.61,49.09,43.70,117377.0,-8963.0,239105.0,-11.62,34.0,X-MC,2.02,72.0,-0.08,1.72,10.89,XY24,BTT,FMCG
43,ITC,452.0,0.37,-1.50,12.47,10.78,29519.0,-3599.0,236719.0,-41.15,47.0,X-LC,2.14,5.0,-0.12,1.70,3.66,X40,NTT,FMCG
78,UNITDSPR,1644.0,0.63,2.84,17.43,20.77,41971.0,6656.0,240800.0,-6.10,43.0,X-MC,2.25,62.0,0.16,1.73,9.96,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COLPAL,3726.84,-0.03,-20.67,78.38,41.51,163759.0,-54435.0,208930.0,-5.42,35.0,X-MC,7.66,61.0,-0.33,1.50,0.09,XY25,ATH,FMCG
53,PAGEIND,51605.08,0.13,-11.28,44.39,28.11,79325.0,-22710.0,178700.0,-35.73,24.0,X-MC,13.79,55.0,-0.29,1.29,0.13,X40,ATH,APPARELS
3,ACC,3906.00,0.22,-26.09,122.08,64.15,214714.0,-62071.0,175880.0,-56.75,32.0,X-SC,3.20,82.0,-0.29,1.27,0.22,XY24,BTT,CEMENT
11,BATAINDIA,2096.00,0.35,-39.15,122.19,35.20,95672.0,-50372.0,78298.0,2.67,32.0,X-SC,16.62,92.0,-0.53,0.56,0.43,X40,NTT,FOOTWEAR
59,RELAXO,1176.00,-0.52,-48.12,193.56,52.31,145776.0,-69847.0,75313.0,-44.73,40.0,X-SC,7.31,91.0,-0.48,0.54,0.81,X40N,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,2.36,-29.06,101.36,42.84,46742.0,-18891.0,46115.0,-53.72,47.0,X-SC,34.74,83.0,-0.40,0.33,4.33,XY24,NTT,MISC
59,RELAXO,1176.00,-0.52,-48.12,193.56,52.31,145776.0,-69847.0,75313.0,-44.73,40.0,X-SC,7.31,91.0,-0.48,0.54,0.81,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.35,-39.15,122.19,35.20,95672.0,-50372.0,78298.0,2.67,32.0,X-SC,16.62,92.0,-0.53,0.56,0.43,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,0.43,-3.35,28.38,24.08,34821.0,-4255.0,122695.0,-11.10,38.0,X-SC,5.59,89.0,-0.12,0.88,15.61,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,0.77,-16.78,72.22,43.32,97887.0,-27332.0,135540.0,-28.43,38.0,X-SC,5.06,90.0,-0.28,0.97,4.07,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,0.44,-9.39,33.22,20.71,113849.0,-35526.0,342711.0,-22.01,70.0,X-LC,3.87,1.0,-0.31,2.46,14.51,X40,ATH,IT
41,INFY,1972.00,0.63,11.45,20.46,34.24,71344.0,35814.0,348702.0,-13.14,68.0,X-LC,4.88,2.0,0.50,2.51,20.68,X40,NTT,IT
75,TMPV,600.00,1.43,-28.11,71.06,22.98,115399.0,-63501.0,162397.0,-25.52,39.0,X-LC,6.84,3.0,-0.55,1.17,2.14,XY24,NTT,AUTO
81,VBL,671.64,-0.67,-4.97,42.77,35.67,128373.0,-15695.0,300147.0,-16.24,50.0,X-LC,3.94,4.0,-0.12,2.16,8.20,X40N,ATH,FMCG
43,ITC,452.00,0.37,-1.50,12.47,10.78,29519.0,-3599.0,236719.0,-41.15,47.0,X-LC,2.14,5.0,-0.12,1.70,3.66,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.44,-38.09,114.64,32.89,54798.0,-29406.0,47800.0,-690.25,51.0,L-MC,6.31,259.0,-0.54,0.34,31.64,XR,NTT,BANKS
6,ASIANTILES,137.00,5.13,-4.26,89.75,81.67,80611.0,-3993.0,89817.0,8083.33,71.0,L-SC,13.01,271.0,-0.05,0.65,73.93,XR,NTT,CERAMICS
50,MASFIN,398.61,1.39,-2.99,25.80,22.05,24524.0,-2925.0,95055.0,-16.44,52.0,H-SC,6.40,164.0,-0.12,0.68,38.17,XR,ATH,FINANCE
13,BSOFT,831.70,-0.66,-21.95,93.19,50.79,102302.0,-30875.0,109778.0,0.12,63.0,H-SC,6.86,151.0,-0.30,0.79,27.82,XR,ATH,IT
39,INDIGOPNTS,1408.00,0.11,-16.18,19.32,0.02,28269.0,-28239.0,146320.0,123.21,42.0,M-SC,13.10,234.0,-1.00,1.05,27.27,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,-0.66,-21.95,93.19,50.79,102302.0,-30875.0,109778.0,0.12,63.0,H-SC,6.86,151.0,-0.30,0.79,27.82,XR,ATH,IT
6,ASIANTILES,137.00,5.13,-4.26,89.75,81.67,80611.0,-3993.0,89817.0,8083.33,71.0,L-SC,13.01,271.0,-0.05,0.65,73.93,XR,NTT,CERAMICS
84,WIPRO,420.00,0.11,8.33,59.01,72.27,108325.0,14122.0,183570.0,-6.99,69.0,M-LC,3.06,99.0,0.13,1.32,15.02,XR,NTT,IT
41,INFY,1972.00,0.63,11.45,20.46,34.24,71344.0,35814.0,348702.0,-13.14,68.0,X-LC,4.88,2.0,0.50,2.51,20.68,X40,NTT,IT
61,ROUTE,2227.21,-1.29,-45.31,209.12,69.05,152177.0,-60296.0,72770.0,-55.87,57.0,H-SC,21.95,140.0,-0.40,0.52,12.23,SR,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.36
1,25,44.82
2,50,76.63


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.90
MC    30.83
LC    24.27
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.67
X40      23.13
X40N     12.97
XR        9.82
XY25      9.08
AR        8.99
OX40N     7.57
SR        1.01
MH        0.76
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.36
H-SC    23.74
X-LC    20.81
M-SC    11.62
X-SC     8.05
H-MC     4.79
L-SC     1.49
M-MC     1.34
M-LC     1.32
H-LC     1.11
L-LC     1.03
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.31,-8.43,43.30
IT,13.20,-14.42,71.96
FINANCE,9.46,-20.96,73.57
MISC,7.26,-28.66,81.28
ELECTRICAL,5.97,-12.92,54.28
PAINTS,5.81,-14.14,31.10
INSURANCE,4.79,-1.67,36.46
PHARMA,4.12,-2.77,35.12
AUTO,2.85,-32.41,78.31


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3213811.0,21
AR,1320765.0,10
XR,1297901.0,13
X40,1006159.0,14
X40N,882046.0,9
OX40N,739405.0,10
XY25,379977.0,6
SR,279528.0,2
MH,79044.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3676810.0,25
M-SC,1444165.0,15
X-MC,1439763.0,16
X-LC,831879.0,11
X-SC,799375.0,8
H-MC,413036.0,3
L-SC,255250.0,3
M-LC,108325.0,1
H-LC,79197.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1292955.0      6
           AR         899518.0      5
M-SC       XY24       826065.0      6
H-SC       XR         789611.0      7
X-MC       X40        494649.0      7
           XY24       409412.0      3
           X40N       347905.0      4
H-SC       OX40N      336154.0      4
X-LC       X40        318278.0      5
M-SC       OX40N      317067.0      5
X-SC       X40N       309866.0      3
           XY24       296277.0      3
H-SC       SR         279528.0      2
X-LC       X40N       224275.0      2
H-MC       AR         217118.0      2
           XY24       195918.0      1
X-SC       X40        193232.0      2
X-LC       XY24       193184.0      2
X-MC       XY25       187797.0      2
M-SC       XR         176101.0      2
L-SC       XR         169066.0      2
M-SC       AR         124932.0      2
M-LC       XR         108325.0      1
X-LC       XY25        96142.0      2
L-SC       OX40N       86184.0      1
H-LC       AR          79197.0      1
H-SC       MH          79044.0      1
M-MC       XY25        59214.0      1
L-MC       XR          54798.0      1
L-LC       XY25        36824.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
